In [9]:
import numpy as np
import pandas as pd
embeddings = {}

df = pd.read_csv("../clean_amlo.csv")



In [34]:
import spacy 
from spacy import displacy 

nlp = spacy.load("es_core_news_lg")
import nltk
import re
from nltk import word_tokenize
from nltk.corpus import stopwords

stop_words_en= stopwords.words('spanish')
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [35]:



def vectorize_clean(texto):
    texto = texto.lower()
    texto = re.sub(r'([^0-9A-Za-z-À-ÿ \t])','', texto)
    texto = word_tokenize(texto)
    texto = [palabra for palabra in texto if palabra not in stop_words_en]
    texto = [lemmatizer.lemmatize(palabra) for palabra in texto]
    texto = ' '.join(texto)
    texto = nlp(texto).vector
    return texto

def vectorize(texto):
    texto = nlp(texto).vector
    return texto

In [10]:
df

,Texto,Clasificacion,clean,vector
0,También vamos a informar este primer día hábil...,apoyo,también ir a informar este primero día hábil d...,[ 1.41899014e+00 2.89622617e+00 -8.29722404e-...
1,Y ahora se va a informar porque se está optand...,apoyo,y ahora él ir a informar porque él estar optar...,[ 1.613179 2.4638274 -1.0321436 0.228727...
2,En esta etapa se combinan los bancos particula...,apoyo,en este etapa él combinar el banco particular ...,[ 1.40079224e+00 2.55894732e+00 -7.46786058e-...
3,"También Ariadna Montiel, que es la secretaria ...",apoyo,también ariadna montiel que ser el secretaria ...,[ 1.7361122e+00 1.4736185e+00 -3.4129769e-02 ...
4,Pero tenerlas terminadas no significa tenerlas...,apoyo,pero tener él terminado no significar tener él...,[ 1.10585928e+00 2.97061157e+00 -1.89105558e+...
...,...,...,...,...
449,"Eso es lo que le puedo decir, con todo respeto...",exterior,ese ser él que él poder decir con todo respeto...,[ 5.83898425e-01 3.60192585e+00 -3.74272203e+...
450,Se va a llegar a concretar porque es que los ...,exterior,él ir a llegar a concretar porque ser que el...,[ 1.98953402e+00 1.27168190e+00 -4.11813051e-...
451,"Entonces, tenemos que hablar mucho con nuestro...",seguridad,entonces tener que hablar mucho con nuestro pa...,[ 1.2499397e+00 2.9624448e+00 -3.3469234e+00 ...
452,Es que ya lo están haciendo. Y a lo mejor el…...,exterior,ser que ya él estar hacer y a él mejor el bu...,[ 1.79775321e+00 2.57321405e+00 -1.54663992e+...


In [37]:
#df['vector'] = df["clean"].apply(vectorize)
#df.head()

In [38]:

df['vector'] = df["vector"].apply(lambda x: 
                           np.fromstring(
                               x.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' '))

In [39]:

def getTopXDocs(frase,x,export=False):
    data = {
        'texto' :[],
        'sims': []
    }
    buscar = vectorize_clean(frase)
    for vector, frase in zip(df["vector"],df["Texto"]):
        A = buscar 
        B = vector
        
        resultado = np.dot(A,B) / (np.linalg.norm(A) * np.linalg.norm(B)) 
        
        data["texto"].append(frase)
        data["sims"].append(resultado)
    final = pd.DataFrame(data).sort_values(by = 'sims', ascending = False).head(x)
    return final

In [40]:
getTopXDocs("hubo muchas muertes", 25)

,texto,sims
159,"Ricolino, sí. Hay que tener mucho cuidado con...",0.503150
156,Ahora que una parte de Televisa se vende a Uni...,0.479012
155,Imagínense cuánto ingresa ahora. Les estoy hab...,0.468538
329,"Entonces, no se permite la corrupción; entonce...",0.468380
419,Otra que podemos tener sobre el incremento de ...,0.464503
441,"Pues no tengo mucha información, pero sí son ...",0.461948
265,"Siempre habla de pie de igualdad, respetando l...",0.456002
407,"Bueno, tenemos muy buena comunicación en el t...",0.452198
246,"Es que no me ha respondido Loret de Mola, no m...",0.451495
188,"No, no estoy de acuerdo en eso; habría que ve...",0.446757


In [11]:
#df = df.drop('Unnamed: 0', axis=1)

In [12]:
#df.to_csv('../clean_amlo.csv', index=False)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Texto          454 non-null    object
 1   Clasificacion  454 non-null    object
 2   clean          454 non-null    object
 3   vector         454 non-null    object
dtypes: object(4)
memory usage: 14.3+ KB
